In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [4]:
from sklearn.metrics import confusion_matrix, classification_report, log_loss

In [5]:
import category_encoders as ce

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
pd.set_option('display.max_columns', 120)

In [94]:
def disp_null_cols(df):
    df_nulls = pd.DataFrame(columns = ['Feature', 'NullCount', 'NullPct'])
    for col in df.columns.to_list():        
        nul_count = df[col].isna().sum()
        if  nul_count > 0:
            df_nulls.loc[len(df_nulls)] = [col, nul_count, np.round(nul_count / df.shape[0], 2)]
            
    return df_nulls.sort_values(by='NullPct', ascending=False)

---

In [134]:
tr = pd.read_csv('train_indessa.csv')

In [135]:
nul_stats = disp_null_cols(tr)

In [136]:
drop_list = nul_stats['Feature'].head().squeeze().to_list()
df_train = tr.drop(drop_list, axis=1)

In [137]:
disp_null_cols(df_train).head(6)

,Feature,NullCount,NullPct
0,batch_enrolled,85149,0.16
13,tot_coll_amt,42004,0.08
14,tot_cur_bal,42004,0.08
15,total_rev_hi_lim,42004,0.08
1,emp_title,30833,0.06
2,emp_length,26891,0.05


In [138]:
df_train[['batch_enrolled']].fillna(' ', inplace=True)
df_train.loc[df_train.batch_enrolled==' ', 'batch_enrolled'] = 'BATCHX'

In [139]:
df_train.dropna(inplace=True)

In [140]:
disp_null_cols(df_train)

,Feature,NullCount,NullPct


In [159]:
num_cols = df_train.select_dtypes(['int', 'float', 'int64']).columns.to_list()
cat_cols = list( set(df_train.columns.to_list()) - set(num_cols) )
incorrect_cols = ['tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'revol_util', 
                  'open_acc', 'total_acc', 'collections_12_mths_ex_med']

In [160]:
for col in incorrect_cols:
    df_train[col] = df_train[col].astype(int)
num_cols = num_cols+incorrect_cols

In [161]:
cat_cols = list (set(cat_cols) - set(incorrect_cols))

In [162]:
df_train[cat_cols].sample(5)

,application_type,home_ownership,initial_list_status,grade,term,emp_length,title,addr_state,sub_grade,last_week_pay,batch_enrolled,emp_title,purpose,verification_status,zip_code,pymnt_plan
119572,INDIVIDUAL,MORTGAGE,w,C,60 months,8 years,Other,SC,C1,57th week,BAT2333412,Chemical Engineer,other,Verified,296xx,n
472100,INDIVIDUAL,RENT,f,B,36 months,6 years,Major purchase,NY,B2,144th week,BAT5341619,Working Media Group,major_purchase,Source Verified,100xx,n
356226,INDIVIDUAL,RENT,f,C,36 months,5 years,Pay Off Bills,TN,C1,87th week,BAT1766061,shipping & recieving/ storeroom,credit_card,Verified,383xx,n
26114,INDIVIDUAL,RENT,w,C,36 months,6 years,Debt consolidation,RI,C5,56th week,BAT2558388,Zoo Keeper,debt_consolidation,Source Verified,029xx,n
235232,INDIVIDUAL,MORTGAGE,w,B,36 months,6 years,Debt consolidation,CA,B2,22th week,BAT3873588,Home Care Provider,debt_consolidation,Verified,925xx,n


In [168]:
df_train.last_week_pay.value_counts()

13th week     23639
9th week      22349
26th week     21427
4th week      20118
22th week     20066
35th week     18627
39th week     18432
17th week     17152
31th week     16867
52th week     14902
65th week     14641
48th week     14558
61th week     13762
78th week     10800
74th week     10038
87th week      8347
NAth week      8163
44th week      8026
91th week      7997
83th week      7297
70th week      7194
43th week      6124
57th week      5847
100th week     5777
109th week     5541
104th week     5344
96th week      4679
117th week     4526
113th week     4478
56th week      3895
156th week     3893
122th week     3707
126th week     3661
135th week     2875
131th week     2499
30th week      2461
139th week     2381
95th week      2011
148th week     1919
144th week     1718
152th week     1646
82th week      1449
18th week      1191
130th week     1059
161th week      930
69th week       929
0th week        666
8th week        468
143th week      370
21th week       320


In [167]:
df_train.last_week_pay.apply(lambda x : int(x.split(' ')[0][:-2]) )

ValueError: invalid literal for int() with base 10: 'NA'